# 大模型文本分类

In [1]:
from langchain_openai.chat_models import ChatOpenAI

In [2]:
llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="empty",
    model="Qwen/Qwen2.5-7B-Instruct"
)

llm.invoke("你好")

AIMessage(content='你好！很高兴见到你。有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 30, 'total_tokens': 43, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'id': 'chatcmpl-714199a07e804a33b6df60085b292440', 'finish_reason': 'stop', 'logprobs': None}, id='run-864dad70-66ea-4b20-9f5a-02d7e0ae453e-0', usage_metadata={'input_tokens': 30, 'output_tokens': 13, 'total_tokens': 43, 'input_token_details': {}, 'output_token_details': {}})

In [4]:
from pydantic import BaseModel, Field

In [5]:
class Classification(BaseModel):
    name: str = Field(description="用户的姓名")
    age: str = Field(description="用户的年龄")

In [ ]:
example = "周杰伦（1979年1月18日—），台湾创作男歌手、演员、词曲作家及制作人。其音乐风行于大中华地区及全球各地的华人社群，并对华语乐坛产生重大影响，也是史上最具影响力及最著名的华语歌手之一[5][6][7]，有“中文流行天王”之美誉[8][9]。"
#! 原理是使用了guided_decoding技术实现的
structured_llm = llm.with_structured_output(Classification, include_raw=False)
structured_llm.invoke(f"请帮我从下列数据中进行提取{example}")

Classification(name='周杰伦', age='1979年1月18日—至今（具体日期未知）')

In [14]:
#! 个人觉得工具调用更加方便
tool_llm = llm.bind_tools(tools=[Classification])
tool_llm.invoke("周杰伦（1979年1月18日—），台湾创作男歌手、演员、词曲作家及制作人。其音乐风行于大中华地区及全球各地的华人社群，并对华语乐坛产生重大影响，也是史上最具影响力及最著名的华语歌手之一[5][6][7]，有“中文流行天王”之美誉[8][9]。")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-34ab704cbd9443ef919a0d4ce7d08220', 'function': {'arguments': '{"name": "周杰伦", "age": "1979年1月18日—"}', 'name': 'Classification'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 268, 'total_tokens': 305, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'id': 'chatcmpl-ea52c5a699164f1f88e151ef7562e4d7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3a2765c9-36a0-4874-87f6-16cde212ee08-0', tool_calls=[{'name': 'Classification', 'args': {'name': '周杰伦', 'age': '1979年1月18日—'}, 'id': 'chatcmpl-tool-34ab704cbd9443ef919a0d4ce7d08220', 'type': 'tool_call'}], usage_metadata={'input_tokens': 268, 'output_tokens': 37, 'total_tokens': 305, 'input_token_details': {}, 'output_token_details': {}})